In [6]:
from classiq import *
from classiq.execution import ExecutionPreferences
from math import pi, log2, ceil
from random import randint, choice
from sympy import isprime

authenticate()

# First round

In [7]:
# Generate random
#max_number = 32
#primes = [i for i in range(max_number) if isprime(i) and i%4==3]
#p = choice(primes)
#q = choice(primes)
#N = p*q
#n = ceil(log2(max(p,q))) # length of x
#m = 2*n # length of y, can be reduced to n+O(1)

# or set prime numbers
p = 3 # first prime number for toy problem
q = 7 # second prime number for toy problem
N = p*q
n = 3
m = 5

In [ ]:
@qfunc
def mcp(ctrl0: QBit, ctrl1: QBit, phi: CReal, trgt: QBit):
    ctrl_arr = QArray("ctrl_arr")
    bind([ctrl0, ctrl1], ctrl_arr)
    control(ctrl=ctrl_arr, stmt_block=lambda: PHASE(phi, trgt))
    bind(ctrl_arr, [ctrl0, ctrl1])

@qfunc
def x2modN_phase(x: QArray[QBit], y: QArray[QBit]):
    for k in range(m):
      for i in range(n):
        phi = 2*pi*2**(i+i+k)/N
        control(ctrl=x[i], stmt_block=lambda:PHASE(phi,y[k]))
    for k in range(m):
      for i in range(n-1):
        for j in range(i+1,n):
          phi = 2*2*pi*2**(i+j+k)/N
          mcp(x[i], x[j], phi, y[k])

@qfunc
def main(x: Output[QArray[QBit]], y: Output[QArray[QBit]]):
    allocate(n, x)
    allocate(m, y)
    hadamard_transform(x)
    hadamard_transform(y)
    x2modN_phase(x, y)
    invert(stmt_block=lambda: qft(y))

qmod_1 = create_model(main)
write_qmod(qmod_1, "x2modN")
qprog_1 = synthesize(qmod_1)
show(qprog_1)
job_1 = execute(qprog_1)
results_1 = job_1.result()
counts_1 = sorted(results_1[0].value.counts.items(), key=lambda item: item[1], reverse=True)[:2**n]
for i in range(2**n):
  x = int(counts_1[i][0][-n:], 2)
  y = round(int(counts_1[i][0][:m], 2)*N/(2**m))
  print('Accept   ' if x**2%N==y else "Reject   ", end = "")
  print(x, 'squared modulo', N, 'is', y)

# Final round

In [14]:
# Choose two input states
x0 = '010'
x1 = '101'
n = len(x0)

# Generate random variables
bitstring = format(randint(0,2**n-1),'b').zfill(n)
angle_sign = randint(0,1)

In [ ]:
@qfunc
def mcx(ctrl0: QBit, ctrl1: QBit, trgt: QBit):
    ctrl_arr = QArray("ctrl_arr")
    bind([ctrl0, ctrl1], ctrl_arr)
    control(ctrl=ctrl_arr, stmt_block=lambda: X(trgt))
    bind(ctrl_arr, [ctrl0, ctrl1])

@qfunc
def inner_product(b: QBit, r: QArray[QBit], x: QArray[QBit]):
    for i in range(n):
      mcx(r[i], x[i], b)

@qfunc
def main(b: Output[QBit], r: Output[QArray[QBit]], x: Output[QArray[QBit]]):
    allocate(n, r)
    inplace_prepare_int(int(bitstring,2), r)
    prepare_ghz_state(n, x)
    X(x[1])
    allocate(1, b)
    inner_product(b, r, x)
    hadamard_transform(x)
    RY((-1)**angle_sign*pi/4, b)

def dot_product(u:str, v:str):
    s=0
    for i in range(len(u)):
      s += int(u[i])*int(v[i])
    return s%2

qmod_2 = create_model(main)
write_qmod(qmod_2, "bell_test")
qmod_2_ep = set_execution_preferences(qmod_2, ExecutionPreferences(num_shots=10000))
qprog_2 = synthesize(qmod_2_ep)
show(qprog_2)
job_2 = execute(qprog_2)
results_2 = job_2.result()
counts_2 = results_2[0].value.counts

r_x0 = dot_product(bitstring, x0)
r_x1 = dot_product(bitstring, x1)
delta_r = abs(r_x0-r_x1)
d = [format(i,'b').zfill(n) for i in range(2**n) if dot_product(format(i,'b').zfill(n),'010')==dot_product(format(i,'b').zfill(n),'101') or delta_r==1]
for i in range(len(d)):
  if delta_r == 0:
    exp = r_x0
  else:
    d_x0 = dot_product(d[i], x0)
    d_x1 = dot_product(d[i], x1)
    delta_d = abs(d_x0-d_x1)
    exp = (delta_d+angle_sign+1)%2
  w = str(d[i])+bitstring+str(exp)
  l = str(d[i])+bitstring+str(abs(exp-1))
  prob = counts_2[w]/(counts_2[w]+counts_2[l])
  print('Accept   ' if prob>0.8 else 'Reject   ', f"{prob:.2%}")